# Prompt enginnering & Backend

## Import and Ingest Data

In [11]:
from dotenv import load_dotenv
import os
import json

In [12]:
load_dotenv()
H2O_API_KEY = os.getenv("H2O_API_KEY") 

In [13]:
from h2ogpte import H2OGPTE

client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key= H2O_API_KEY,
)

# Create a new collection
collection_id = client.create_collection(
    name='devMeetingAssistant',
    description='Test development for meeting assistant (prompt engineering)',
)

In [14]:
with open('Data/fake_sample.txt', 'rb') as f:
    meeting_data1 = client.upload('Data/fake_sample.txt', f)
client.ingest_uploads(collection_id, [meeting_data1,])

Job(id='c517bb2a-467d-4285-8796-d942309dc1f6', name='', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 19, 15, 23, 36, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='e846ed9e50e544479bc13d750ee29b36', status='Collecting done.'), JobStatus(id='625aa094262e4c0e9c99b12a3e75ada6', status='Collecting done.'), JobStatus(id='435545a3ef2c4a778ebf946395630b03', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 19, 15, 23, 40, tzinfo=TzInfo(UTC)), duration='4s')

## Start Session

In [15]:
chat_session_id = client.create_chat_session(collection_id)

In [8]:
# llm_list = client.get_llms() # default use first elem, index[0]
# for llm in llm_list:
#     print(llm)

{'base_model': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'prompt_type': 'mixtral', 'prompt_dict': {'promptA': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'promptB': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'PreInstruct': '<s> [INST] ', 'PreInput': None, 'PreResponse': '[/INST]', 'terminate_response': ['[INST]', '</s>'], 'chat_sep': ' ', 'chat_turn_sep': '</s> ', 'humanstr': '[INST]', 'botstr': '[/INST]', 'generates_leading_space': False, 'system_prompt': 'You are an AI that follows instructions extremely well and as helpful as possible.', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None,

In [16]:
def rag_chat(client, chat_session_id, main_prompt, system_prompt):
    with client.connect(chat_session_id) as session:
        answer = session.query(
            message=main_prompt,
            system_prompt=system_prompt,
            rag_config={
            "rag_type": "rag", # https://h2oai.github.io/h2ogpte/getting_started.html#advanced-controls-for-document-q-a
            },
            llm="gpt-3.5-turbo-0613",
        )
        return answer

## Prompt 1: General summary

In [18]:
system_prompt = """
You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
You will be given meeting transcript(s) from real project meetings held over zoom.
Follow all the instructions closely or everyone in the team will be fired.
If you do well, you will be rewarded with a $1000 tip.
"""
main_prompt = """
Execute all the following steps: \
1. Examine the document and identify all names of the attendees, save this list of names in your memory
2. Identify and extract the main purpose or agenda of the meeting
3. Identify all actionables mentioned during the meeting
4. Extract all the actionables and action plans, identify all details related to it including the deadlines, who it is assigned to, etc.
5. Summarize the entire transcript
6. Present all identified information in the following JSON format:
{"Agenda": "", "Meeting Summary":""} for the entire meeting
{"Actionables":[{"Action":"", "Deadline":"", "Assigned":""}]} for each action plan
"""
meeting_summary_response = rag_chat(client, chat_session_id, main_prompt, system_prompt)
print(meeting_summary_response.content)

{"Agenda": "Review progress on the project, discuss roadblocks, and plan next steps.", "Meeting Summary": "The meeting started with Eugene introducing the agenda and asking for confirmation from the attendees. Ryan provided an update on completing the research phase and starting the design proposal. Ben shared that the budget allocation has been finalized and submitted for approval. Micole mentioned a minor issue with software integration but believed it could be resolved with more testing. The discussion then moved to potential roadblocks, with Ryan suggesting consulting a design specialist for guidance. Ben was assigned to schedule a meeting with the design specialist. Micole mentioned no immediate issues impacting the timeline. The action plan for the upcoming week included completing the design proposal by Ryan and confirming the meeting with the design specialist by Ben.", "Actionables": [{"Action": "Complete the design proposal", "Deadline": "Wednesday", "Assigned": "Ryan"}, {"Ac

In [ ]:
system_prompt = """
You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
You will be given meeting transcript(s) from real project meetings held over zoom.
Follow all the instructions closely or everyone in the team will be fired.
If you do well, you will be rewarded with a $1000 tip.
"""
main_prompt = """
Execute all the following steps: \
1. Examine the document and identify all names of the attendees, save this list of names in your memory
2. Identify and extract the main purpose or agenda of the meeting
3. Identify all actionables mentioned during the meeting
4. Extract all the actionables and action plans, identify all details related to it including the deadlines, who it is assigned to, etc.
5. Summarize the entire transcript
6. Present all identified information in the following JSON format:
{"Agenda": "", "Meeting Summary":""} for the entire meeting
{"Actionables":[{"Action":"", "Deadline":"", "Assigned":""}]} for each action plan
"""
meeting_summary_response = rag_chat(client, chat_session_id, main_prompt, system_prompt)
print(meeting_summary_response.content)

## Prompt 2: Sentiment Analysis

In [22]:
sentiment_prompt = f"""
Execute all the following steps: \
1. Examine the contexts of this JSON object: {meeting_summary_response}
2. For each actionable, execute the following steps
a. identify the sentence in the document where it is mentioned
b. Interpret the sentence, based on this context, categorize the priority of this task as "High", "Medium" or "Low"
c. Edit the JSON object, add a column to the dictionary for this action as follows: "Priority": ""
3. Present all identified information in the following JSON format:
{{"Agenda": "", "Meeting Summary":""}} for the entire meeting
{{"Actionables":[{{"Action":"", "Deadline":"", "Assigned":"", "Priority":""}}]}} for each action plan
"""

sentiment_reply = rag_chat(client, chat_session_id, sentiment_prompt, system_prompt)
print(sentiment_reply.content)

{
  "Agenda": "Review progress on the project, discuss roadblocks, and plan next steps.",
  "Meeting Summary": "The meeting started with Eugene introducing the agenda and asking for confirmation from the attendees. Ryan provided an update on completing the research phase and starting the design proposal. Ben shared that the budget allocation has been finalized and submitted for approval. Micole mentioned a minor issue with software integration but believed it could be resolved with more testing. The discussion then moved to potential roadblocks, with Ryan suggesting consulting a design specialist for guidance. Ben was assigned to schedule a meeting with the design specialist. Micole mentioned no immediate issues impacting the timeline. The action plan for the upcoming week included completing the design proposal by Ryan and confirming the meeting with the design specialist by Ben.",
  "Actionables": [
    {
      "Action": "Complete the design proposal",
      "Deadline": "Wednesday",


In [24]:
sentiment_reply.content

'{\n  "Agenda": "Review progress on the project, discuss roadblocks, and plan next steps.",\n  "Meeting Summary": "The meeting started with Eugene introducing the agenda and asking for confirmation from the attendees. Ryan provided an update on completing the research phase and starting the design proposal. Ben shared that the budget allocation has been finalized and submitted for approval. Micole mentioned a minor issue with software integration but believed it could be resolved with more testing. The discussion then moved to potential roadblocks, with Ryan suggesting consulting a design specialist for guidance. Ben was assigned to schedule a meeting with the design specialist. Micole mentioned no immediate issues impacting the timeline. The action plan for the upcoming week included completing the design proposal by Ryan and confirming the meeting with the design specialist by Ben.",\n  "Actionables": [\n    {\n      "Action": "Complete the design proposal",\n      "Deadline": "Wedne

In [26]:
# Parse the JSON-like string
data = json.loads(sentiment_reply.content)

# Extract information from the parsed data
agenda = data.get('Agenda', '')
meeting_summary = data.get('Meeting Summary', '')
actionables = data.get('Actionables', [])

# Format the post-meeting email
email_subject = f"Post-Meeting Summary: {agenda}"
email_body = f"Dear Team,\n\nHere's a summary of our recent meeting:\n\nAgenda: {agenda}\n\nMeeting Summary:\n{meeting_summary}\n\nActionables:\n"
for action in actionables:
    email_body += f"- {action['Action']} (Assigned to: {action['Assigned']}, Deadline: {action['Deadline']}, Priority: {action['Priority']})\n"

email_body += "\nPlease let me know if anything needs clarification or if there are additional action items to add.\n\nBest regards,\n[Your Name]"
print(email_body)

Dear Team,

Here's a summary of our recent meeting:

Agenda: Review progress on the project, discuss roadblocks, and plan next steps.

Meeting Summary:
The meeting started with Eugene introducing the agenda and asking for confirmation from the attendees. Ryan provided an update on completing the research phase and starting the design proposal. Ben shared that the budget allocation has been finalized and submitted for approval. Micole mentioned a minor issue with software integration but believed it could be resolved with more testing. The discussion then moved to potential roadblocks, with Ryan suggesting consulting a design specialist for guidance. Ben was assigned to schedule a meeting with the design specialist. Micole mentioned no immediate issues impacting the timeline. The action plan for the upcoming week included completing the design proposal by Ryan and confirming the meeting with the design specialist by Ben.

Actionables:
- Complete the design proposal (Assigned to: Ryan, D

In [ ]:
# # Alternatively, use another conversation
# chat_session_id_2 = client.create_chat_session(collection_id)

# sentiment_reply_alt = rag_chat(client, chat_session_id_2, sentiment_prompt, system_prompt)
# print(sentiment_reply.content)

## Prompt 3: Language Translation

In [ ]:
## havent thought about it yet lol

TODO:  
  
Output email .txt  
input user name for email sign-off?  
Encapsulate all functions  
Language Translation prompt  

In [ ]:
# with open("trial_meeting_minutes_output.txt","w") as f:
#     f.write(llm_reply.content)

# Emailing stuff that I havent looked at yet

## Convert output file into pdf file to attach to email

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def create_pdf(text, filename='output.pdf'):
    # Create a canvas
    c = canvas.Canvas(filename, pagesize=letter)
    
    # Set up font
    c.setFont("Helvetica", 12)
    
    # Split text into lines
    lines = text.split('\n')
    
    # Set starting y-coordinate
    y = 750
    
    # Write text to the canvas
    for line in lines:
        c.drawString(50, y, line)
        y -= 15  # Adjust for next line
        
    # Save the canvas
    c.save()


In [ ]:
# Create PDF
create_pdf(llm_reply.content, "trial_meeting_minutes_output.pdf")

Automate Email sending

https://keentolearn.medium.com/automating-emails-with-python-a-comprehensive-guide-ba00fa98b92#:~:text=Once%20you%20have%20set%20up,account%2C%20and%20send%20the%20email.

## Created junk email for everyone to use
email : gpt10.4213.1@outlook.com <br>
pw: gpt10email<br>

name : gpt<br>
surname : 10<br>
country : senegal<br>
dob : 7 Mar 1989

#### Send email without attachment

In [ ]:
# #This works!
# import smtplib

# smtp_server = 'smtp-mail.outlook.com'
# smtp_port = 587
# smtp_username = 'gpt10.4213.1@outlook.com'
# smtp_password = 'gpt10email'

# from_email = 'gpt10.4213.1@outlook.com'
# to_email = 'gpt10.4213.1@outlook.com'
# subject = 'Hello, world!'
# body = 'This is a test email. using smtplib~~'

# message = f'Subject: {subject}\n\n{body}'

# with smtplib.SMTP(smtp_server, smtp_port) as smtp:
#     smtp.starttls()
#     smtp.login(smtp_username, smtp_password)
#     smtp.sendmail(from_email, to_email, message)

In [ ]:
smtp_server_mapping = {
    "gmail":"smtp.gmail.com",
    "outlook":"smtp-mail.outlook.com",
    "hotmail":"smtp.office365.com",
    "yahoo":"smtp.mail.yahoo.com",
}  #the rest dont care first since different smtp_port # ....

#### Send email with attachment

works with a list of email addresses ~ish

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

smtp_server = 'smtp-mail.outlook.com'
smtp_port = 587
smtp_username = 'gpt10.4213.1@outlook.com'
smtp_password = 'gpt10email'

from_email = 'gpt10.4213.1@outlook.com'
to_email = 'gpt10.4213.1@outlook.com'
to_email_list = ['gpt10.4213.1@outlook.com',]

meeting_date = "5/3/2024"
subject = f"{meeting_date} Meeting"  # meeting_date either user input or LLM search through document.
body = f'Please find attached the report. \n This is a summary of the meeting. \n {llm_reply2.content}'

msg = MIMEMultipart()
msg['From'] = from_email
# msg['To'] = to_email # send to single user
msg['To'] = ", ".join(to_email_list) # send to multiple users
msg['Subject'] = subject
msg.attach(MIMEText(body))

with open('trial_meeting_minutes_output.pdf', 'rb') as f:
    attachment = MIMEApplication(f.read(), _subtype='pdf')
    attachment.add_header('Content-Disposition', 'attachment', filename='report.pdf')
    msg.attach(attachment)

with open('trial_meeting_minutes_output.txt', 'rb') as f:
    attachment = MIMEApplication(f.read(), _subtype='txt')
    attachment.add_header('Content-Disposition', 'attachment', filename='report_friendly.txt')
    msg.attach(attachment)

with smtplib.SMTP(smtp_server, smtp_port) as smtp:
    smtp.starttls()
    smtp.login(smtp_username, smtp_password)
    smtp.send_message(msg)